In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 35
n_classes = 10

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Tomatoes",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 16011 files belonging to 10 classes.


In [4]:
class_names = dataset.class_names
print(class_names)

['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [5]:
def get_dataset_partitions_tf(
    ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000
):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    if shuffle:
        ds.shuffle(shuffle_size, seed=22)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# caches images for next epoch, prefetch fetches next batch before next iteration

In [7]:
resize_and_rescale = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
    ]
)

In [8]:
data_augmentation = tf.keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
            layers.experimental.preprocessing.RandomRotation(0.2),
        ]
    )

In [9]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [10]:
model = models.Sequential(
    [
        resize_and_rescale,
        data_augmentation,
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            padding="same",
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(n_classes, activation="softmax"),
    ]
)

In [11]:
model.build(input_shape=input_shape)

In [14]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9,
   )

model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=32,
    verbose=1,
    validation_data=val_ds,
)

Epoch 1/35
400/400 [==============================] - 70s 175ms/step - loss: 2.1993 - accuracy: 0.2020 - val_loss: 2.2111 - val_accuracy: 0.1750
Epoch 2/35
400/400 [==============================] - 69s 173ms/step - loss: 2.1979 - accuracy: 0.2009 - val_loss: 2.2208 - val_accuracy: 0.1750
Epoch 3/35
400/400 [==============================] - 71s 177ms/step - loss: 2.1977 - accuracy: 0.2020 - val_loss: 2.2204 - val_accuracy: 0.1750
Epoch 4/35
346/400 [========================>.....] - ETA: 8s - loss: 2.1973 - accuracy: 0.2020

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
model.save("../model/3")